In [2]:
# Dependencies
import pandas as pd
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [38]:
# Read in candidates data
df1 = pd.read_csv("candidates_clean.csv")
df_cand = df1.copy()
df_cand.head()

,Unnamed: 0,id,cycle,fec_cand_id,cid,first_last_party,party,dist_id_run_for,dist_id_currently_held,current_candidate,cycle_candidate,crpico,recip_code,nopacs,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized,result
0,0,56756,1996,P80001118,N00000948,Lenora B Fulani (3),3,PRES,,,,C,3L,,0.0,6000.0,6000,0.0,L
1,1,58068,1996,S6MN00143,N00004451,Stephen Bonsal Young (R),R,MNS1,,,Y,C,RL,,492.0,97408.0,264254,166354.0,L
2,2,57382,1996,S6DE00073,N00001670,Raymond J Clatworthy (R),R,DES2,,Y,Y,C,RL,,202815.0,618935.0,1636414,814664.0,L
3,3,57290,1996,S4VT00066,N00000525,Gavin Terence Mills (I),U,VTS1,,,,C,3L,,0.0,266.0,266,0.0,L
4,4,53386,1996,H2OK05045,N00005551,Ernest J Istook (R),R,OK05,OK05,Y,Y,I,RW,,130384.0,99600.0,399980,169996.0,W


In [34]:
# Read in pac data 
df2 = pd.read_csv("pacs.csv", on_bad_lines='skip')
df_pac = df2.copy()
df_pac.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,cycle,fec_rec_no,pac_id,cid,amount,date,real_code,type,di,fec_cand_id
0,1,2000,1047706,C00147066,N00005892,0,2000-04-12 00:00:00,F3300,24K,D,H4TX22023
1,2,2000,1470335,C00139964,N00007879,0,2000-08-01 00:00:00,A1500,24K,D,S0WA00019
2,3,2000,1469635,C00139964,N00006246,0,2000-08-01 00:00:00,A1500,24K,D,S4WY00055
3,4,2000,0364522,C00089086,N00002898,1,1999-07-20 00:00:00,H1700,24Z,D,H4FL16021
4,5,2000,1786857,C00089086,N00005638,1,2000-09-12 00:00:00,H1700,24Z,D,H4TX26016


## Clean

In [44]:
df_pac.isnull().sum()

id              0
cycle           0
fec_rec_no      0
pac_id          2
cid            90
amount          6
date           19
real_code      84
type           14
di             15
fec_cand_id    18
dtype: int64

In [46]:
df_pac = df_pac.dropna()

id             0
cycle          0
fec_rec_no     0
pac_id         0
cid            0
amount         0
date           0
real_code      0
type           0
di             0
fec_cand_id    0
dtype: int64

In [53]:
df_pac.duplicated().sum()

0

In [49]:
df_pac = df_pac.drop_duplicates()

In [50]:
# df_pac.to_csv('pacs_clean.csv')

In [ ]:
# Join pac and candidates dfs
df_paccand = df_cand.merge(df_pac, how='left', on='cid')
df_paccand.head()

## Explore Features

In [29]:
# See if cycle_candidate is significant
# if there are W and blank c_c, then it's significant
# if there L for every blank c_c, insignificant - drop
df2 = df_cand[(df_cand['cycle_candidate'] == ' ') & (df_cand['result'] == 'W')]
df2.head(10)

# KEEP for now, but test both ways.

,Unnamed: 0,id,cycle,fec_cand_id,cid,first_last_party,party,dist_id_run_for,dist_id_currently_held,current_candidate,cycle_candidate,crpico,recip_code,nopacs,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized,result
13961,13961,45306,1992,P80000912,N00008954,Al Gore (D),D,PRES,TNS2,,,C,DW,,25363.0,11012.0,36375,0.0,W
15708,15708,9170,2002,P80000235,N00000245,John Kerry (D),D,MAS2,MAS2,,,I,DW,Y,2669345.0,6022491.0,8691836,0.0,W
13960,13960,45283,1992,P80000581,N00005037,Richard A Gephardt (D),D,MO03,MO03,,,I,DW,,1250509.0,1626733.0,4745956,1868714.0,W
13963,13963,45555,1992,S2CA00302,N00007571,Robert T Matsui (D),D,CA05,CA03,,,I,DW,,797133.0,141000.0,1228521,290388.0,W
15611,15611,62485,1998,S0IN00061,N00003875,Baron Hill (D),D,IN09,,,,O,DW,,574168.0,423431.0,997599,0.0,W
4823,4823,57803,1996,S6NY00052,N00000968,Daniel Patrick Moynihan (D),D,NYS1,NYS1,,,I,DW,,-809.0,13145.0,55884,43548.0,W
4803,4803,57830,1996,S6OR00144,N00007710,Elizabeth Furse (D),D,OR01,OR01,,,I,DW,,548481.0,447859.0,1895789,899449.0,W
15614,15614,62542,1998,S2GA00027,N00002526,Bob Barr (R),R,GA07,GA07,,,I,RW,,307376.0,526765.0,834141,0.0,W
16024,16024,42845,1992,H2CA06028,N00006692,Barbara Boxer (D),D,CAS1,CA06,,,O,DW,,3308168.0,3837467.0,7145635,0.0,W
13856,13856,39955,1990,P80000581,N00005037,Richard A Gephardt (D),D,MO03,MO03,,,I,DW,,953508.0,741736.0,2510583,815339.0,W


In [39]:
# Choose columns for features
columns = ['party', 'dist_id_run_for', 'dist_id_currently_held', 'current_candidate', 'cycle_candidate', 'crpico', 'nopacs', 'raised_from_pacs', 'raised_from_individuals',	'raised_total',	'raised_unitemized', 'pac_id', 'amount', 'type']

# consider dropping later:
  #cycle_candidate
  #nopacs
  #dist_id_currently_held


,Unnamed: 0,id_x,cycle_x,fec_cand_id_x,cid,first_last_party,party,dist_id_run_for,dist_id_currently_held,current_candidate,cycle_candidate,crpico,recip_code,nopacs,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized,result,id_y,cycle_y,fec_rec_no,pac_id,amount,date,real_code,type,di,fec_cand_id_y
0,0,56756,1996,P80001118,N00000948,Lenora B Fulani (3),3,PRES,,,,C,3L,,0.0,6000.0,6000,0.0,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,58068,1996,S6MN00143,N00004451,Stephen Bonsal Young (R),R,MNS1,,,Y,C,RL,,492.0,97408.0,264254,166354.0,L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,57382,1996,S6DE00073,N00001670,Raymond J Clatworthy (R),R,DES2,,Y,Y,C,RL,,202815.0,618935.0,1636414,814664.0,L,248704.0,2002,1815670,C90007436,3300,2002-11-04 00:00:00,Y0000,24E,I,S6DE00073
3,2,57382,1996,S6DE00073,N00001670,Raymond J Clatworthy (R),R,DES2,,Y,Y,C,RL,,202815.0,618935.0,1636414,814664.0,L,257799.0,2002,1831149,C00003418,23565,2002-10-31 00:00:00,Z5100,24C,I,S6DE00073
4,2,57382,1996,S6DE00073,N00001670,Raymond J Clatworthy (R),R,DES2,,Y,Y,C,RL,,202815.0,618935.0,1636414,814664.0,L,257800.0,2002,1831150,C00003418,23565,2002-10-31 00:00:00,Z5100,24C,I,S6DE00073
